In [1]:
from kafka import KafkaConsumer
from pymongo import MongoClient
import re
from tqdm import tqdm
from time import sleep
from json import loads

In [2]:
common_regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-)'
combined_regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-) "(?P<referrer>[^"]*)" "(?P<useragent>[^"]*)'
columns = ['client', 'userid', 'datetime', 'method', 'request', 'status', 'size', 'referer', 'user_agent']

In [16]:
topic='tst'

In [17]:
consumer = KafkaConsumer(
    topic,
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
    value_deserializer=lambda x: x.decode('utf-8')
    )

In [18]:
myclient = MongoClient("mongodb://localhost:27017/")
dblist = myclient.list_database_names()
if "mydatabase" in dblist:
  print("The database exists.")
else:
    mydb = myclient["mydatabase"]

The database exists.


In [19]:
mydb = myclient["mydatabase"]
mycol = mydb["logs"]

In [20]:
def preprocess(line):
    d=dict()
    for i,col in enumerate(columns):
        d[col]=line[0][i]
    return d

In [22]:
d=dict()
i=0
n=mycol.count_documents({})
print(n)
for message in consumer:
    if i<n:
        i+=1
        continue
    log_line = re.findall(combined_regex, message.value)
    try:
        d=preprocess(log_line)
    except:
        continue    
    mycol.insert_one(d)
    print('{} added to {}'.format(d, mycol))
    sleep(4)

1478
{'client': '66.249.66.194', 'userid': '-', 'datetime': '22/Jan/2019:04:07:25 +0330', 'method': 'GET', 'request': '/filter/null%2Ct399?page=0&phrase=%D9%85%D8%A7%DB%8C+%D8%B3%D8%A7%D9%86&o=t', 'status': '302', 'size': '0', 'referer': '-', 'user_agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)', '_id': ObjectId('61fa436f465a700d605d10c4')} added to Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mydatabase'), 'logs')
{'client': '130.185.74.243', 'userid': '-', 'datetime': '22/Jan/2019:04:07:25 +0330', 'method': 'GET', 'request': '/filter/stexists,p6446,t164,b262', 'status': '200', 'size': '32557', 'referer': '-', 'user_agent': 'Mozilla/5.0 (Windows NT 6.1; rv:42.0) Gecko/20100101 Firefox/42.0', '_id': ObjectId('61fa4373465a700d605d10c5')} added to Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mydatabase'), 'logs')
{'client':

{'client': '204.18.72.194', 'userid': '-', 'datetime': '22/Jan/2019:04:07:27 +0330', 'method': 'GET', 'request': '/image/131/brand', 'status': '200', 'size': '2599', 'referer': 'https://www.zanbil.ir/m/filter/b1%2Cp5%2Cb2', 'user_agent': 'Mozilla/5.0 (Linux; Android 5.1.1; SAMSUNG SM-J500H Build/LMY48B) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/3.3 Chrome/38.0.2125.102 Mobile Safari/537.36', '_id': ObjectId('61fa43ab465a700d605d10d3')} added to Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mydatabase'), 'logs')
{'client': '204.18.72.194', 'userid': '-', 'datetime': '22/Jan/2019:04:07:27 +0330', 'method': 'GET', 'request': '/image/29/productTypeType', 'status': '200', 'size': '10813', 'referer': 'https://www.zanbil.ir/m/filter/b1%2Cp5%2Cb2', 'user_agent': 'Mozilla/5.0 (Linux; Android 5.1.1; SAMSUNG SM-J500H Build/LMY48B) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/3.3 Chrome/38.0.2125.102 Mobile Safar

{'client': '204.18.72.194', 'userid': '-', 'datetime': '22/Jan/2019:04:07:29 +0330', 'method': 'GET', 'request': '/image/55651/productModel/200x200', 'status': '200', 'size': '3867', 'referer': 'https://www.zanbil.ir/m/filter/b1%2Cp5%2Cb2', 'user_agent': 'Mozilla/5.0 (Linux; Android 5.1.1; SAMSUNG SM-J500H Build/LMY48B) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/3.3 Chrome/38.0.2125.102 Mobile Safari/537.36', '_id': ObjectId('61fa43e7465a700d605d10e2')} added to Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mydatabase'), 'logs')
{'client': '204.18.72.194', 'userid': '-', 'datetime': '22/Jan/2019:04:07:29 +0330', 'method': 'GET', 'request': '/image/62658/productModel/200x200', 'status': '200', 'size': '2573', 'referer': 'https://www.zanbil.ir/m/filter/b1%2Cp5%2Cb2', 'user_agent': 'Mozilla/5.0 (Linux; Android 5.1.1; SAMSUNG SM-J500H Build/LMY48B) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/3.3 Chrome/38

{'client': '207.46.13.136', 'userid': '-', 'datetime': '22/Jan/2019:04:07:32 +0330', 'method': 'GET', 'request': '/filter/b1,b103,b105,b109,b111,b113,b135,b144,b176,b186,b192,b198,b221,b261,b291,b454,b52,b598,b741,b77,b83', 'status': '200', 'size': '39243', 'referer': '-', 'user_agent': 'Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)', '_id': ObjectId('61fa4423465a700d605d10f1')} added to Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mydatabase'), 'logs')
{'client': '54.36.148.241', 'userid': '-', 'datetime': '22/Jan/2019:04:07:32 +0330', 'method': 'GET', 'request': '/filter/b131,b3,b862,p5935', 'status': '200', 'size': '34579', 'referer': '-', 'user_agent': 'Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)', '_id': ObjectId('61fa4427465a700d605d10f2')} added to Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mydataba

KeyboardInterrupt: 